<a href="https://www.kaggle.com/code/nguyenthien3001/employee-attrition?scriptVersionId=244103259" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Thêm data

In [1]:
!pip install -q -U scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 62.9 MB/s eta 0:00:00:00:01:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
category-encoders 2.7.0 requires scikit-learn<1.6.0,>=1.0.0, but you have scikit-learn 1.7.0 which is incompatible.


In [2]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/employee-attrition-prediction-dataset/employee_attrition_dataset_10000.csv
/kaggle/input/employee-attrition-prediction-dataset/employee_attrition_dataset.csv


In [3]:
# Import library
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from imblearn.metrics import classification_report_imbalanced
from sklearn.naive_bayes import *
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,roc_auc_score
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

ImportError: cannot import name '_safe_tags' from 'sklearn.utils._tags' (/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py)

In [ ]:
df = pd.read_csv("/kaggle/input/employee-attrition-prediction-dataset/employee_attrition_dataset_10000.csv")
df.head()

In [ ]:
df.info()

In [ ]:
df['Attrition'] = df['Attrition'].apply(lambda x: 0 if x == 'No' else 1)

In [ ]:
df['Attrition'].value_counts().plot(kind="bar")

# Train

In [ ]:
# Train nhu binh thuong
dataset = df.drop(['Employee_ID','Years_in_Current_Role','Years_Since_Last_Promotion','Absenteeism'], axis=1)

In [ ]:
dataset_numeric_columns = dataset.select_dtypes(include=['int', 'float']).columns.drop(['Attrition'])

In [ ]:
dataset_numeric_columns

In [ ]:
# Tao ra onehot
dataset_dummy = pd.get_dummies(dataset,drop_first=True)


X = dataset_dummy.drop(["Attrition"],axis=1)
y = dataset_dummy['Attrition']

X_train, X_test, y_train, y_test = train_test_split(X, y , test_size = 0.2, random_state=42)

scaler = MinMaxScaler()
scaler.fit(X_train[dataset_numeric_columns])
X_train[dataset_numeric_columns]= scaler.transform(X_train[dataset_numeric_columns])
X_test[dataset_numeric_columns]= scaler.transform(X_test[dataset_numeric_columns])


#Performance evaluation
def print_scores(y_true, y_pred):
    print(classification_report_imbalanced(y_true, y_pred, digits = 4))
    print(accuracy_score(y_true, y_pred, digits = 4))

## Chạy trên dữ liệu thô

In [ ]:
# Danh sách mô hình Naive Bayes
models = {
    "BernoulliNB": BernoulliNB(),
    "ComplementNB": ComplementNB(),
    "GaussianNB": GaussianNB(),
    "MultinomialNB": MultinomialNB()
}

# Train và đánh giá từng mô hình
for name, model in models.items():
    print(f"Training model: {name}")
    model.fit(X_train, y_train)  # Train model
    y_pred = model.predict(X_test)  # Dự đoán
    
    print(f"Results for {name}:")
    print_scores(y_test, y_pred)  # In kết quả
    print("=" * 50)  # Phân cách output

## Chạy trên dữ liệu undersampling

In [ ]:
from imblearn.under_sampling import EditedNearestNeighbours
under = EditedNearestNeighbours(n_neighbors = 5)

#dataset_dummy = pd.get_dummies(dataset,drop_first=True)

#X = dataset_dummy.drop(["Attrition"],axis=1)
#y = dataset_dummy['Attrition']

X_under, y_under = under.fit_resample(X,y)
X_under.shape, y_under.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_under, y_under , test_size = 0.2, random_state=42)

scaler = MinMaxScaler()
scaler.fit(X_train[dataset_numeric_columns])
X_train[dataset_numeric_columns]= scaler.transform(X_train[dataset_numeric_columns])
X_test[dataset_numeric_columns]= scaler.transform(X_test[dataset_numeric_columns])

# Danh sách mô hình Naive Bayes
models = {
    "BernoulliNB": BernoulliNB(),
    "ComplementNB": ComplementNB(),
    "GaussianNB": GaussianNB(),
    "MultinomialNB": MultinomialNB()
}

# Train và đánh giá từng mô hình
for name, model in models.items():
    print(f"Training model: {name}")
    model.fit(X_train, y_train)  # Train model
    y_pred = model.predict(X_test)  # Dự đoán
    
    print(f"Results for {name}:")
    print_scores(y_test, y_pred)  # In kết quả
    print("=" * 50)  # Phân cách output

## OverSampling SMOTE

In [ ]:

# SMOTE
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state = 42)

#dataset_dummy = pd.get_dummies(dataset,drop_first=True)

#X = dataset_dummy.drop(["Attrition"],axis=1)
#y = dataset_dummy['Attrition']

X_sm, y_sm = sm.fit_resample(X, y)
X_sm.shape, y_sm.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size = 0.2, random_state=42)

scaler = MinMaxScaler()
scaler.fit(X_train[dataset_numeric_columns])
X_train[dataset_numeric_columns]= scaler.transform(X_train[dataset_numeric_columns])
X_test[dataset_numeric_columns]= scaler.transform(X_test[dataset_numeric_columns])

# Danh sách mô hình Naive Bayes
models = {
    "BernoulliNB": BernoulliNB(),
    "ComplementNB": ComplementNB(),
    "GaussianNB": GaussianNB(),
    "MultinomialNB": MultinomialNB()
}

# Train và đánh giá từng mô hình
for name, model in models.items():
    print(f"Training model: {name}")
    model.fit(X_train, y_train)  # Train model
    y_pred = model.predict(X_test)  # Dự đoán
    
    print(f"Results for {name}:")
    print_scores(y_test, y_pred)  # In kết quả
    print("=" * 50)

## Hybrid SMOTENN

In [ ]:
# SMOTE
from imblearn.combine import SMOTEENN

smnn = SMOTEENN(random_state = 42, enn = EditedNearestNeighbours(n_neighbors = 5))

#dataset_dummy = pd.get_dummies(dataset,drop_first=True)

#X = dataset_dummy.drop(["Attrition"],axis=1)
#y = dataset_dummy['Attrition']

X_smnn, y_smnn = smnn.fit_resample(X, y)
X_smnn.shape, y_smnn.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_smnn, y_smnn, test_size = 0.2, random_state=42)

scaler = MinMaxScaler()
scaler.fit(X_train)
X_train= scaler.transform(X_train)
X_test= scaler.transform(X_test)

# Danh sách mô hình Naive Bayes
models = {
    "BernoulliNB": BernoulliNB(),
    "ComplementNB": ComplementNB(),
    "GaussianNB": GaussianNB(),
    "MultinomialNB": MultinomialNB()
}

# Train và đánh giá từng mô hình
for name, model in models.items():
    print(f"Training model: {name}")
    model.fit(X_train, y_train)  # Train model
    y_pred = model.predict(X_test)  # Dự đoán
    
    print(f"Results for {name}:")
    print_scores(y_test, y_pred)  # In kết quả
    print("=" * 50)